code in functions
#super function not working

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt

## Load Data
- registrations
- courses
- students
- student_vle
- vle
- student_assessments
- assessments

In [3]:
zipfile_ou = '../data/anonymisedData.zip'
prediction_point = None

In [4]:


def load_data(zip_file_path):
    '''Loads the data from the Open University Learning Analytics dataset zip file.'''
    
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        registrations = pd.read_csv(zip_file.open('studentRegistration.csv'))
        courses = pd.read_csv(zip_file.open('courses.csv'))
        students = pd.read_csv(zip_file.open('studentInfo.csv'))
        student_vle = pd.read_csv(zip_file.open('studentVle.csv'))
        vle = pd.read_csv(zip_file.open('vle.csv'))
        student_assessments = pd.read_csv(zip_file.open('studentAssessment.csv'))
        assessments = pd.read_csv(zip_file.open('assessments.csv'))
    
    return registrations, courses, students, student_vle, vle, student_assessments, assessments






In [94]:
registrations, courses, students, student_vle, vle, student_assessments, assessments = load_data(zipfile_ou)

In [ ]:
#reg_after_wd.head(20)

## 19 students withdrew on start date.

In [95]:
prediction_point = 30

## Clean and Add Student Information
- studentInfo
- registrations

In [8]:
def student_data(students, courses, registrations, prediction_point=None):
    '''Returns a dataframe of student data based on `students`: 

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge students and courses
    - merge registrations
    - drop missing value rows (date_registration, imd_band)
    - drop students who unregistered before registering
    - fills missing date_unregistration with module_presentation_length - that is, assumes they completed the course
    - removes students who withdrew or failed before prediction point
    '''

    # merge students and courses
    student_regist = pd.merge(students, courses, on=['code_module', 'code_presentation'], validate='many_to_one')

    # merge registrations
    student_regist = pd.merge(student_regist, registrations, on=['code_module', 'code_presentation', 'id_student'], how = 'left', validate='1:1')

    #prediction_point default
    #if prediction_point is None:
    #    prediction_point = max(student_regist['module_presentation_length']) + 7

    # drop missing value rows (date_registration, imd_band)
    student_regist.dropna(subset=['date_registration', 'imd_band'], inplace=True)

    # set default status to 'keep'
    student_regist['status'] = 'keep'

    # prediction point must be less than course length, integer, and greater than 0
    if prediction_point is not None:
        assert isinstance(prediction_point, int) and prediction_point > 0 and prediction_point <= (max(student_regist['module_presentation_length']) + 7), "Error: Invalid prediction point.\n\nPlease provide an integer value greater than 0 and less than or equal to the maximum course length.\n\nThis is the number of days from the start of the course for which you want to predict the outcome."
    
        # withdrawn or failed before prediction point - remove
        withdrawn_fail_condition = (student_regist['final_result'].isin(['Withdrawn', 'Fail'])) & (student_regist['date_unregistration'] <= prediction_point)
        student_regist.loc[withdrawn_fail_condition, 'status'] = 'remove_outcome_known'
    
        # unregister after prediction point - keep
        unregister_after_condition = student_regist['date_unregistration'] > prediction_point
        student_regist.loc[unregister_after_condition, 'status'] = 'keep'
    
        # if no unregistration date - keep
        no_unregistration_condition = student_regist['date_unregistration'].isna()
        student_regist.loc[no_unregistration_condition, 'status'] = 'keep'
    
        # query case
        student_regist.loc[~(withdrawn_fail_condition | unregister_after_condition | no_unregistration_condition), 'status'] = 'query'


    # rows which need investigation
    query_rows = student_regist[student_regist['status'] == 'query'] | student_regist[student_regist['status'].isna()]

    # print rows which need investigation
    if not query_rows.empty:
        print("The following rows need investigation.  They are excluded from the following analysis: \n")
        print(query_rows)
        student_regist = student_regist[~student_regist.isin(query_rows)].dropna()

    # replace missing date_unreg with module_presentation_length
    student_regist['date_unregistration'] = student_regist['date_unregistration'].fillna(student_regist['module_presentation_length'])

    # drop students who unregistered before starting
    student_regist = student_regist[student_regist['date_unregistration'] >= student_regist['date_registration']]

    # remove rows from final student df which are not needed
    model_final = student_regist[student_regist['status'] != 'remove_outcome_known']
   

    return model_final



In [96]:
model_final = student_data(students, courses, registrations, prediction_point)

In [97]:
model_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26432 entries, 0 to 32592
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 26432 non-null  object 
 1   code_presentation           26432 non-null  object 
 2   id_student                  26432 non-null  int64  
 3   gender                      26432 non-null  object 
 4   region                      26432 non-null  object 
 5   highest_education           26432 non-null  object 
 6   imd_band                    26432 non-null  object 
 7   age_band                    26432 non-null  object 
 8   num_of_prev_attempts        26432 non-null  int64  
 9   studied_credits             26432 non-null  int64  
 10  disability                  26432 non-null  object 
 11  final_result                26432 non-null  object 
 12  module_presentation_length  26432 non-null  int64  
 13  date_registration           26432 no

## Clean and Add Assessment Information
- assessmentInfo
- student assessments

In [17]:
def add_assessments(student_assessments, assessments, courses, model_final, prediction_point=None):
    '''Returns updated model_final dataframe with student assessment data added:

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge 'assessments' and 'courses' and 'student_assessments'
    - populate missing 'date' values with course final week - as suggested in literature
    - remove students from 'model_final' who have no score for an assessment
    - remove students with odd assessment dates (before registration, after unregistration, or very far into future)
    - remove students who withdrew or failed before prediction point
    - calculate new features - average score, submission date distance, proportion of assessments submitted
    '''
    # merge 'assessments' and 'courses' on 'code_module' and 'code_presentation'
    course_assess = pd.merge(assessments,courses, on=['code_module', 'code_presentation'], how='left')

    # fill in the missing 'date' values with course final week (as per literature)
    value_to_fill = course_assess['module_presentation_length'] - 3
    course_assess['date'] = course_assess['date'].fillna(value_to_fill)

    # merge student_assessments with course_assess
    stu_assess = pd.merge(student_assessments, course_assess, on=['id_assessment'], how='left')

    # drop students who have no score for an assessment
    missing_score_rows = stu_assess[stu_assess['score'].isna()]

    if not missing_score_rows.empty:
        print("\n\nThe following students have missing 'scores'. They are excluded from the analysis: \n")
        print(missing_score_rows)

        # student ids with missing score
        unique_ids_missing = missing_score_rows['id_student'].unique()

        # remove students with missing score from model_final
        model_final = model_final[~model_final['id_student'].isin(unique_ids_missing)]

        # drop rows with missing score
        stu_assess.dropna(subset=['score'], inplace=True)

    # remove students with negative or extreme date_submitted values
    max_module_length = stu_assess['module_presentation_length'].max() +10
    greater_than_max_length = stu_assess[stu_assess['date_submitted'] > max_module_length]['id_student'].unique()
    less_than_zero = stu_assess[stu_assess['date_submitted'] < 0]['id_student'].unique()
    students_to_remove = set(greater_than_max_length) | set(less_than_zero)
    model_final = model_final[~model_final['id_student'].isin(students_to_remove)]
    stu_assess = stu_assess[~stu_assess['id_student'].isin(students_to_remove)]


    
    # if prediction_point is None
    if prediction_point is None:
        # no data reduction
        model_student_assess = stu_assess
        model_course_assess = course_assess
    else:
        # prediction point must be less than course length, integer, and greater than 0
        assert isinstance(prediction_point, int) and prediction_point > 0 and prediction_point <= (max(courses['module_presentation_length'])+7), "Error: Invalid prediction point.\n\nPlease provide an integer value greater than 0 and less than or equal to the maximum course length.\n\nThis is the number of days from the start of the course for which you want to predict the outcome."

        # reduce data by prediction point
        model_student_assess = stu_assess[stu_assess['date'] <= prediction_point]
        model_course_assess = course_assess[course_assess['date'] <= prediction_point]

    # expected assessment details
    expected_ass = model_course_assess.groupby(['code_module', 'code_presentation'])['id_assessment'].count().reset_index()
    expected_ass = expected_ass.rename(columns={'id_assessment': 'exp_sub_count'})
    date_sum = model_course_assess.groupby(['code_module', 'code_presentation'])['date'].sum().reset_index()
    expected_ass = expected_ass.merge(date_sum, on=['code_module', 'code_presentation'], how='left')
    expected_ass = expected_ass.rename(columns={'date': 'exp_sub_date_sum'})

    # summarise students' assessments by module_presentation
    student_assessment_summary = model_student_assess.groupby(['id_student', 'code_module', 'code_presentation']).agg(
        count_id_assessment=('id_assessment', 'count'),
        sum_score=('score', 'sum'),
        sum_date=('date', 'sum')
    ).reset_index()

    # merge student_assessment_summary and expected_ass on code_module and code_presentation
    merged_assess_summary = student_assessment_summary.merge(expected_ass, on=['code_module', 'code_presentation'])

    # calculate the new features
    merged_assess_summary['prop_submissions'] = merged_assess_summary['count_id_assessment'] / merged_assess_summary['exp_sub_count']
    merged_assess_summary['avg_score'] = merged_assess_summary['sum_score'] / merged_assess_summary['exp_sub_count']
    merged_assess_summary['submission_distance'] = merged_assess_summary['exp_sub_date_sum'] - merged_assess_summary['sum_date']

    # merge with 'model_final'
    model_final = model_final.merge(merged_assess_summary, on=['id_student', 'code_module', 'code_presentation'], how='inner')

    return model_final

In [98]:
model_final = add_assessments(student_assessments, assessments, courses, model_final, prediction_point)



The following students have missing 'scores'. They are excluded from the analysis: 

        id_assessment  id_student  date_submitted  is_banked  score   
215              1752      721259              22          0    NaN  \
937              1754      260355             127          0    NaN   
2364             1760     2606802             180          0    NaN   
3358            14984      186780              77          0    NaN   
3914            14984      531205              26          0    NaN   
...               ...         ...             ...        ...    ...   
148929          34903      582670             241          0    NaN   
159251          37415      610738              87          0    NaN   
166390          37427      631786             221          0    NaN   
169725          37435      648110              62          0    NaN   
170103          37435      480914              49          0    NaN   

       code_module code_presentation assessment_type   date 

In [99]:
model_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18645 entries, 0 to 18644
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 18645 non-null  object 
 1   code_presentation           18645 non-null  object 
 2   id_student                  18645 non-null  int64  
 3   gender                      18645 non-null  object 
 4   region                      18645 non-null  object 
 5   highest_education           18645 non-null  object 
 6   imd_band                    18645 non-null  object 
 7   age_band                    18645 non-null  object 
 8   num_of_prev_attempts        18645 non-null  int64  
 9   studied_credits             18645 non-null  int64  
 10  disability                  18645 non-null  object 
 11  final_result                18645 non-null  object 
 12  module_presentation_length  18645 non-null  int64  
 13  date_registration           186

## Clean and Add VLE Information
- vleInfo
- student vle

In [20]:
def add_vle_data(model_final, student_vle, vle, courses, prediction_point=None):
    '''Returns updated model_final dataframe with vle data:

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge 'vle' and 'courses' and 'student_vle'
    - remove columns (week_from, week_to)
    - filter the rows where 'date' <= 'prediction_point'
    - create new features: vle_activity_count, student vle type count, student total clicks, student days active (engaged)
    
    '''
    # merge 'vle' and 'courses' on 'code_module' and 'code_presentation'
    course_vle = vle.merge(courses, on=['code_module', 'code_presentation'], how='left').drop(['week_from', 'week_to'], axis=1)

    # merge vle with student_vle
    all_stu_vle = pd.merge(student_vle, course_vle, on=['id_site', 'code_module', 'code_presentation'], how='left')

    #prediction_point default
    #if prediction_point is None:
    #    prediction_point = max(courses['module_presentation_length'])
    
    # if prediction_point is None
    if prediction_point is None:
        # no filtering
        all_stu_vle = all_stu_vle
    else:
        # prediction point must be less than or equal to the maximum date
        assert isinstance(prediction_point, int) and prediction_point <= max(all_stu_vle['date']), "Error: Invalid prediction point.\n\nPlease provide an integer value less than or equal to the maximum date.\n\nThis is the cutoff date for filtering the rows."

        # filter the rows where 'date' <= 'prediction_point'
        all_stu_vle = all_stu_vle[all_stu_vle['date'] <= prediction_point]


    # filter the rows where 'date' is greater than 'module_presentation_length'
    vle_after_done = all_stu_vle[all_stu_vle['date'] > all_stu_vle['module_presentation_length']]

    if not vle_after_done.empty:
        print("The following rows need investigation. They are excluded from the following analysis: \n")
        print(vle_after_done)

        # match rows based on 'code_module', 'code_presentation', and 'id_student'
        matching_rows = model_final[model_final[['code_module', 'code_presentation', 'id_student']].isin(vle_after_done).all(axis=1)]

        # remove the matching rows from 'model_final'
        model_final = model_final[~model_final.index.isin(matching_rows.index)]

    # aggregations for each column
    aggregations = {
        'id_site': 'count',
        'activity_type': 'nunique',
        'sum_click': 'sum',
        'date': lambda x: x.nunique()
    }

    # group and apply the aggregations
    grouped_stu_vle = all_stu_vle.groupby(['code_module', 'code_presentation', 'id_student']).agg(aggregations).reset_index()

    # rename the columns
    grouped_stu_vle.rename(columns={
        'id_site': 'stu_activity_count',
        'activity_type': 'stu_activity_type_count',
        'sum_click': 'stu_total_clicks',
        'date': 'stu_days_active'
    }, inplace=True)

    # number of vle types per module_presentation
    group_vle = vle.groupby(['code_module', 'code_presentation'])['activity_type'].nunique().reset_index()
    group_vle.rename(columns={'activity_type': 'mod_pres_vle_type_count'}, inplace=True)

    # merge 'grouped_stu_vle' and 'group_vle' on 'code_module' and 'code_presentation'
    merged_vle_summary = grouped_stu_vle.merge(group_vle, on=['code_module', 'code_presentation'], how='left')

    # merge with 'model_final'
    model_final = model_final.merge(merged_vle_summary, on=['id_student', 'code_module', 'code_presentation'], how='inner')

    return model_final


In [89]:
model_final = add_vle_data(model_final, student_vle, vle, courses, prediction_point)

In [90]:
model_final.count()

code_module                   19947
code_presentation             19947
id_student                    19947
gender                        19947
region                        19947
highest_education             19947
imd_band                      19947
age_band                      19947
num_of_prev_attempts          19947
studied_credits               19947
disability                    19947
final_result                  19947
module_presentation_length    19947
date_registration             19947
date_unregistration           19947
status                        19947
count_id_assessment           19947
sum_score                     19947
sum_date                      19947
exp_sub_count                 19947
exp_sub_date_sum              19947
prop_submissions              19947
avg_score                     19947
submission_distance           19947
stu_activity_count            19947
stu_activity_type_count       19947
stu_total_clicks              19947
stu_days_active             

In [23]:
def tidy_up_model_final(model_final):
    '''Returns updated model_final dataframe:
    - deleted unnecessary columns
    - reordered columns
    - added subject, year, month columns
    
    '''
    # new columns for module_presentation - subject, year, month
    model_final['year'] = model_final['code_presentation'].str[:4].astype(int)
    model_final['month'] = model_final['code_presentation'].str[-1].map({'J': 'Oct', 'B': 'Feb'})

    # module subject mapping
    code_module_mapping = {
        'AAA': 'SocSci',
        'BBB': 'SocSci',
        'GGG': 'SocSci',
        'CCC': 'Stem',
        'DDD': 'Stem',
        'EEE': 'Stem',
        'FFF': 'Stem'
    }
    model_final['subject'] = model_final['code_module'].map(code_module_mapping)

    # rename 'module_presentation_length' to 'course_length'
    model_final.rename(columns={'module_presentation_length': 'course_length'}, inplace=True)

    # combine 'code_module', 'code_presentation', and 'id_student' into 'mod_pres_stu'
    #model_final['mod_pres_stu'] = model_final['code_module'] + '-' + model_final['code_presentation'] + '-' + model_final['id_student'].astype(str)

    # drop the separate columns 'code_module', 'code_presentation', and 'id_student'
    #model_final.drop(columns=['code_presentation'], inplace=True)

    # move 'final_result' to the last column
    final_result_column = model_final.pop('final_result')
    model_final['final_result'] = final_result_column

    # insert 'mod_pres_stu' as the first column
    #mod_pres_stu_column = model_final.pop('mod_pres_stu')
    #model_final.insert(0, 'mod_pres_stu', mod_pres_stu_column)

    # drop columns: 'count_id_assessment', 'sum_score', 'sum_date', 'exp_sub_count', 'exp_sub_date_sum'
    model_final.drop(columns=['count_id_assessment', 'sum_score', 'sum_date', 'exp_sub_count', 'exp_sub_date_sum', 'id_student'], inplace=True)
    
    # used for EDA - keeping student id
    #model_final.drop(columns=['count_id_assessment', 'sum_score', 'sum_date', 'exp_sub_count', 'exp_sub_date_sum'], inplace=True)

    return model_final


In [91]:
model_final = tidy_up_model_final(model_final)

In [92]:
model_final.count()

code_module                19947
code_presentation          19947
gender                     19947
region                     19947
highest_education          19947
imd_band                   19947
age_band                   19947
num_of_prev_attempts       19947
studied_credits            19947
disability                 19947
course_length              19947
date_registration          19947
date_unregistration        19947
status                     19947
prop_submissions           19947
avg_score                  19947
submission_distance        19947
stu_activity_count         19947
stu_activity_type_count    19947
stu_total_clicks           19947
stu_days_active            19947
mod_pres_vle_type_count    19947
year                       19947
month                      19947
subject                    19947
final_result               19947
dtype: int64

In [81]:
model_final.head()

#duplicate_students = model_final[model_final.duplicated(subset='mod_pres_stu', keep=False)]['mod_pres_stu'].unique()

#duplicate_students

#missing_values = model_final.isnull().sum()
#print("Missing values:\n", missing_values, "\n")

,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,...,submission_distance,stu_activity_count,stu_activity_type_count,stu_total_clicks,stu_days_active,mod_pres_vle_type_count,year,month,subject,final_result
0,AAA,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,...,0.0,135,6,713,31,9,2013,Oct,SocSci,Pass
1,AAA,2013J,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,...,0.0,349,7,1121,65,9,2013,Oct,SocSci,Pass
2,AAA,2013J,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,...,0.0,529,8,1730,98,9,2013,Oct,SocSci,Pass
3,AAA,2013J,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,...,0.0,262,7,863,54,9,2013,Oct,SocSci,Pass
4,AAA,2013J,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,...,0.0,465,7,1547,95,9,2013,Oct,SocSci,Pass


In [93]:
model_final.to_csv('../data/20230524_final_model_210days.csv', index=False)


In [80]:
# not working

#def process_data(zip_file, prediction_point):
    # Load data from the zip file
    #registrations, courses, students, student_vle, vle, student_assessments, assessments = load_data(zip_file)

    # Process student data
   # model_final = student_data(students, courses, registrations, prediction_point)

    # Add assessments to model_final
   # model_final = add_assessments(model_final, student_assessments, assessments, courses, prediction_point)

    # Add VLE data to model_final
   # model_final = add_vle_data(model_final, student_vle, vle, courses, prediction_point)

    # Tidy up model_final
   # model_final = tidy_up_model_final(model_final)

  #  return model_final




In [ ]:
# not working

#model_final = process_data(zipfile_ou, prediction_point)